In [17]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

In [15]:
float(np.random.rand(1))

0.8951119925011539

In [9]:
!unzip -q lego_city_dataset_forward_2019-03-15_21-01-35.zip

In [2]:
NBINS = 21

In [18]:
def get_steering(path):
    return (float(int(path[9:12])) - 50.0) / 50.0

def get_gaussian(mean, stdev, nbins):
    x = np.linspace(-1.0, 1.0, num=nbins)
    y = np.exp(-(x - mean)**2 / (stdev**2))
    return y

def get_bin(value, nbins):
    return np.digitize(value, np.linspace(-1.0, 1.0, num=nbins))

class SteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, stdev=0.3, nbins=21):
        self.nbins = nbins
        self.stdev = stdev
        self.directory = directory
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        steering = float(get_steering(os.path.basename(image_path)))
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()
    
    
class MergedSteeringDataset(torch.utils.data.Dataset):
    
    def __init__(self, all_dir, task_dir, flipped=False):
        self.flipped = flipped
        self.all_dir = all_dir
        self.task_dir = task_dir
        self.all_image_paths = glob.glob(os.path.join(self.all_dir, '*.jpg'))
        self.task_image_paths = glob.glob(os.path.join(self.task_dir, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.all_image_paths) + len(self.task_image_paths)
    
    def __getitem__(self, idx):
        flip_all = False
        if idx < len(self.all_image_paths):
            image_path = self.all_image_paths[idx]
            
            if float(np.random.rand(1)) > 0.5:
                flip_all = True
        else:
            image_path = self.task_image_paths[idx - len(self.all_image_paths)]
        
        image = PIL.Image.open(image_path)
        steering = float(get_steering(os.path.basename(image_path)))
        
        if self.flipped:
            image = transforms.functional.hflip(image)
            steering = -steering
        
        if flip_all:
            image = transforms.functional.hflip(image)
            steering = -steering
            
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        #target = get_gaussian(steering, self.stdev, self.nbins)
        #target = get_bin(steering, self.nbins)
        
        return image, torch.tensor([steering]).float()

In [20]:
dataset = MergedSteeringDataset('dataset_all', 'dataset_right', flipped=False)

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

In [21]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)
device = torch.device('cuda')
model = model.to(device)

# train regression:

In [22]:
NUM_EPOCHS = 60
BEST_MODEL_PATH = 'best_steering_model_right_merged.pth'
best_loss = 1e9

#optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

0.626841, 0.768184
0.084167, 0.338674
0.046371, 0.038911
0.024569, 0.031243
0.018465, 0.016597
0.016692, 0.027060
0.023692, 0.034505
0.012317, 0.010997
0.008204, 0.102105
0.015391, 0.009672
0.010738, 0.017054
0.013146, 0.011873
0.012604, 0.020328
0.013090, 0.012819
0.013647, 0.010928
0.013019, 0.017836
0.010607, 0.018431
0.007052, 0.007699
0.007029, 0.007391
0.006412, 0.008054
0.010974, 0.008271
0.005069, 0.009363
0.018893, 0.033706
0.008549, 0.025291
0.006550, 0.006439
0.005896, 0.009165
0.003996, 0.010436
0.006265, 0.005211
0.005118, 0.015055
0.006258, 0.006032
0.003506, 0.007355
0.004647, 0.011738
0.006839, 0.008705
0.009377, 0.012228
0.008872, 0.010145
0.008494, 0.020101
0.008707, 0.008721
0.007771, 0.010352
0.005282, 0.015519
0.006680, 0.032582
0.012041, 0.090892
0.008586, 0.020881
0.011095, 0.012493
0.009631, 0.022652
0.006769, 0.009988
0.003582, 0.010311
0.007742, 0.010416
0.008045, 0.029259
0.009974, 0.018516
0.006367, 0.010252
0.002974, 0.012123
0.007546, 0.015420
0.017131, 0.

In [7]:
torch.save(model.state_dict(), BEST_MODEL_PATH)

In [23]:
torch.save(model.state_dict(), 'best_steering_model_right_merged_end.pth')